# Enriquecimento de clientes

Nesse notebook vamos utilizar os dados criados na etapa anterior para enriquecer os clientes com *features* que serão utilizadas no algoritmo de Machine Learning. Escolhemos as seguintes *features*:

- **data de última compra**: clientes que estão há muito tempo sem comprar tem maior chance de estarem inadimplentes
- **valor médio de compras nos últimos 4, 8 e 12 meses**
- **média do número de itens de produto comprados nos últimos 4, 8 e 12 meses**

O valor médio da compra e o número de itens comprados nos últimos 4, 8 e 12 meses são importantes para descrever o grau de atividade do cliente na empresa.

## Preparando o ambiente

O código abaixo adiciona a **raiz** do projeto, que contém códigos e dados necessários para o "Hands on".

In [1]:
root = '/home/bigdata/jupyterhub'

import sys
sys.path.append(root)

wd = '/delta'

O trecho de código abaixo prepara o ambiente, carregando códigos auxiliares e dados de configuração.O código disponível no pacote *commom.utils* na classe *DataframeUtils* contém vários métodos que facilitam a leitura e escrita dos dados do Postgres. A classe *DataframeUtils* também inicia uma instância do Apache Spark com o Delta Lake integrado ao Spark.

Já o arquivo *config.yaml* tem os dados de acesso ao Postgres e Kafka.

In [2]:
import yaml

from common.utils import DataframeUtils
import pyspark.sql.functions as F

config = yaml.safe_load(open('../config.yaml'))
dfu = DataframeUtils(config)

# Spark session
spark = dfu.spark()

Os trechos abaixo criam os Dataframes com os dados de clientes e pedidos.

In [3]:
clientes = spark.read. \
              format('delta'). \
              load(f'{wd}/data/clientes-silver')

In [4]:
pedidos = spark.read. \
              format('delta'). \
              load(f'{wd}/data/pedidos-silver')

### Exercício

Adicione o código que cria o Dataframe **itens** para armazenar os itens de pedido.

In [5]:
# código para criar o dataframe de itens de pedidos

itens = spark.read. \
              format('delta'). \
              load(f'{wd}/data/itens-silver')

## Última compra de cada cliente

O código abaixo agrega os dados de pedidos para obter a data de última compra de cada cliente utilizando a API do Spark. Os dados são agregados por cliente (groupby) e obtida o maior valor da data de compra (order_date), definindo o nome da coluna para *ultima_compra* com o *alias("ultima_compra")*. Estes dados são armazenados no DataFrame **ultima_compra_df**. Após este passo, uma junção (left join) é realizada entre o DataFrame **clientes** e o DataFrame **ultima_compra_df** para enriquecer clientes com dados de última compra. Os clientes enriquecidos ficarão armazenados no DataFrame **clientes_enriquecidos**.

In [6]:
ultima_compra_df = pedidos.groupby("client_id") \
                    .agg(F.max("order_date").alias("ultima_compra"))
clientes_enriquecidos = clientes.join(ultima_compra_df, "client_id", "left")
clientes_enriquecidos.printSchema()

root
 |-- client_id: string (nullable = true)
 |-- key: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- cnae_id: string (nullable = true)
 |-- defaulting: boolean (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- ultima_compra: date (nullable = true)



# Valor médio de compras de pedidos

O trecho de código abaixo calcula o preço médio dos pedidos nos últimos 4 meses. O código é bem similiar ao enriquecimento de clientes. Os dados de pedidos são filtrados para obter apenas os pedidos realizados (*order_date*) nos últimos 4 meses ou 120 dias. Os dados filtrados são agregados por cliente (groupby) e obtida a média do valor de compra (*order_amount*) com o valor arredondado para duas casas decimais. O código define o nome *pedidos_4_meses* para a coluna que representa o valor médio de compras dos últimos 4 meses. Os dados são armazenados no DataFrame **avg_order_4m_df**. Após este passo, uma junção (left join) é realizada entre o DataFrame **clientes** e o DataFrame **avg_order_4m_df** para enriquecer clientes com dados do valor médio de compras dos últimos 4 meses. Os clientes enriquecidos ficarão armazenados no DataFrame **clientes_enriquecidos**.

### Pedidos: Valor médio de compra (últimos 4 meses)

In [7]:
avg_order_4m_df = pedidos.filter("order_date >= date_sub(current_date, 120)") \
                             .groupby("client_id") \
                             .agg(F.round(F.avg("order_amount"), 2).alias("pedidos_4_meses"))

clientes_enriquecidos = clientes_enriquecidos.join(avg_order_4m_df, "client_id", "left")
clientes_enriquecidos.printSchema()

root
 |-- client_id: string (nullable = true)
 |-- key: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- cnae_id: string (nullable = true)
 |-- defaulting: boolean (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- ultima_compra: date (nullable = true)
 |-- pedidos_4_meses: double (nullable = true)



### Exercício - Pedidos: Valor médio de compra (últimos 8 meses)

Altere o filtro de dados e realizar a consulta para calcular o valor médio de pedidos dos últimos 8 meses. Enriqueça o DataFrame **clientes_enriquecidos** com uma nova coluna **pedidos_8_meses** com o valor médio de compra dos últimos 8 meses.

In [8]:
avg_order_8m_df = pedidos.filter("order_date >= date_sub(current_date, 240)") \
                             .groupby("client_id") \
                             .agg(F.round(F.avg("order_amount"), 2).alias("pedidos_8_meses"))

clientes_enriquecidos = clientes_enriquecidos.join(avg_order_8m_df, "client_id", "left")
clientes_enriquecidos.printSchema()

root
 |-- client_id: string (nullable = true)
 |-- key: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- cnae_id: string (nullable = true)
 |-- defaulting: boolean (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- ultima_compra: date (nullable = true)
 |-- pedidos_4_meses: double (nullable = true)
 |-- pedidos_8_meses: double (nullable = true)



### Exercício - Pedidos: Valor médio de compra (últimos 12 meses)

Altere o filtro de dados e realizar a consulta para calcular o valor médio de pedidos dos últimos 12 meses. Enriqueça o DataFrame **clientes_enriquecidos** com uma nova coluna **pedidos_12_meses** com o valor médio de compra dos últimos 12 meses.

In [9]:
avg_order_12m_df = pedidos.filter("order_date >= date_sub(current_date, 360)") \
                             .groupby("client_id") \
                             .agg(F.round(F.avg("order_amount"), 2).alias("pedidos_12_meses"))
clientes_enriquecidos = clientes_enriquecidos.join(avg_order_12m_df, "client_id", "left")
clientes_enriquecidos.printSchema()

root
 |-- client_id: string (nullable = true)
 |-- key: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- cnae_id: string (nullable = true)
 |-- defaulting: boolean (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- ultima_compra: date (nullable = true)
 |-- pedidos_4_meses: double (nullable = true)
 |-- pedidos_8_meses: double (nullable = true)
 |-- pedidos_12_meses: double (nullable = true)



# Exercício - Quantidade média de itens pedidos

Calcule o preço médio dos itens de pedidos dos últimos 4, 8 e 12 meses. Veja o código feito para pedidos e tente realizar o mesmo para itens de pedidos.

**Dica**: a coluna **items_count** representa o número de itens vendidos em cada compra.

### Exercício - Quantidade média de itens de pedidos (4 meses)

Enriqueça o DataFrame **clientes_enriquecidos** com uma nova coluna **itens_4_meses** com a quantidade média de itens de pedidos vendidos nos últimos 4 meses.

In [10]:
avg_itens_4m_df = itens.filter("order_date >= date_sub(current_date, 120)") \
                             .groupby("client_id") \
                             .agg(F.round(F.avg("items_count"), 2).alias("itens_4_meses"))
clientes_enriquecidos = clientes_enriquecidos.join(avg_itens_4m_df, "client_id", "left")
clientes_enriquecidos.printSchema()

root
 |-- client_id: string (nullable = true)
 |-- key: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- cnae_id: string (nullable = true)
 |-- defaulting: boolean (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- ultima_compra: date (nullable = true)
 |-- pedidos_4_meses: double (nullable = true)
 |-- pedidos_8_meses: double (nullable = true)
 |-- pedidos_12_meses: double (nullable = true)
 |-- itens_4_meses: double (nullable = true)



### Exercício - Quantidade média de itens de pedidos (8 meses)

Enriqueça o DataFrame **clientes_enriquecidos** com uma nova coluna **itens_8_meses** com a quantidade média de itens de pedidos vendidos nos últimos 8 meses.

In [11]:
avg_itens_8m_df = itens.filter("order_date >= date_sub(current_date, 240)") \
                             .groupby("client_id") \
                             .agg(F.round(F.avg("items_count"), 2).alias("itens_8_meses"))
clientes_enriquecidos = clientes_enriquecidos.join(avg_itens_8m_df, "client_id", "left")
clientes_enriquecidos.printSchema()

root
 |-- client_id: string (nullable = true)
 |-- key: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- cnae_id: string (nullable = true)
 |-- defaulting: boolean (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- ultima_compra: date (nullable = true)
 |-- pedidos_4_meses: double (nullable = true)
 |-- pedidos_8_meses: double (nullable = true)
 |-- pedidos_12_meses: double (nullable = true)
 |-- itens_4_meses: double (nullable = true)
 |-- itens_8_meses: double (nullable = true)



### Exercício - Quantidade média de itens de pedidos (12 meses)

Enriqueça o DataFrame **clientes_enriquecidos** com uma nova coluna **itens_12_meses** com a quantidade média de itens de pedidos vendidos nos últimos 12 meses.

In [12]:
avg_itens_12m_df = itens.filter("order_date >= date_sub(current_date, 360)") \
                             .groupby("client_id") \
                             .agg(F.round(F.avg("items_count"), 2).alias("itens_12_meses"))
clientes_enriquecidos = clientes_enriquecidos.join(avg_itens_12m_df, "client_id", "left")
clientes_enriquecidos.printSchema()

root
 |-- client_id: string (nullable = true)
 |-- key: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- cnae_id: string (nullable = true)
 |-- defaulting: boolean (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- ultima_compra: date (nullable = true)
 |-- pedidos_4_meses: double (nullable = true)
 |-- pedidos_8_meses: double (nullable = true)
 |-- pedidos_12_meses: double (nullable = true)
 |-- itens_4_meses: double (nullable = true)
 |-- itens_8_meses: double (nullable = true)
 |-- itens_12_meses: double (nullable = true)



# Exercício - Escrita na Tabela Gold

Nesta última etapa, o DataFrame *clientes_enriquecidos* é escrito na tabela gold no formato **Delta** em "/data/gold". Modifique o código para salvar também no formato **Parquet** em "/data/gold-parquet".

In [13]:
clientes_enriquecidos. \
  write. \
  mode('overwrite'). \
  format('delta'). \
  save(f'{wd}/data/gold')

In [14]:
clientes_enriquecidos. \
  write. \
  mode('overwrite'). \
  format('parquet'). \
  save(f'{wd}/data/gold-parquet')

# Próxima etapa

Agora os dados estão prontos para serem utilizados nos algoritmos de aprendizado de máquina com o objetivo de predizer se o cliente está inadimplente. 

Você acha que poderia adicionar mais alguma *feature* importante para o algoritmo de aprendizado de máquina?